## Minimal Agent

#### Architecture
- Default ReAct from LangGraph (for function error handling)

#### Tools
- database
- visualization
- UNESCO API
- Excel reader (for Limitations)

#### Prompt Adjustments
- give examples for final output
    - closed questions: output should be answer to text + limitations (e.g. for South Africa, plus comparison with benchmark?)
    - open questions: answer with numbers + paragraphs individual explanation + limitations
    - visualization questions: visualization + interpretation
    - out of scope questions: playful decline + explanation of pirls + link

#### State Management
- control input and output of tools (JSON as input possible?)

#### Tool Adjustments
- switch to controlled visualization
- try out statistics function
- try out image generation

In [1]:
!pip install -r ../requirements.txt

In [2]:
import dotenv
assert dotenv.load_dotenv()

In [3]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
MODEL_ID5 = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
MODEL_ID4 = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

HEADERS = {
    "anthropic-beta": "max-tokens-3-5-sonnet-2024-07-15",
    "Content-Type": "application/json"
}



# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0, "max_tokens": 16384, 'top_p': 0.9, 'top_k': 100})
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID4, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [4]:
import tools.csv_handling as csv_tools
import tools.database as db_tools
import tools.web_crawl as web_tools
import tools.data_viz as viz_tools
import tools.pdf_handling as pdf_tools
import tools.stats as stats_tools

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
tools_researcher = [db_tools.query_database, db_tools.get_possible_answers_to_question, db_tools.get_questions_of_given_type]

tools_chart = [viz_tools.create_quickchart_url]

tools_web = [web_tools.get_unesco_data, web_tools.crawl_subpages, web_tools.scrape_text] # 

tools_file = [csv_tools.csv_to_json_string, csv_tools.process_first_sheet_to_json_from_url, csv_tools.calculate_pearson_multiple, csv_tools.extract_table_from_url_to_string_with_auto_cleanup]

tools_pdf = [pdf_tools.retrieve_from_pickle]

tools_stats = [stats_tools.analyze_pirls_data]

tools = tools_researcher + tools_chart + tools_web + tools_file # tools_pdf

In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [8]:
class SQLAgent:
    def __init__(self, model, tools, system_prompt=""):
        self.system_prompt = system_prompt
        
        # initialising graph with a state 
        graph = StateGraph(AgentState)
        
        # adding nodes 
        graph.add_node("llm", self.call_llm)
        graph.add_node("function", self.execute_function)
        graph.add_conditional_edges(
            "llm",
            self.exists_function_calling,
            {True: "function", False: END}
            )
        graph.add_edge("function", "llm")
        
        # setting starting point
        graph.set_entry_point("llm")
        
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_function_calling(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def execute_function(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    
    def run(self, initial_messages):
        try:
            # Execute the graph and get the final state
            final_state = self.graph.invoke({"messages": [HumanMessage(content=initial_messages)]}, {"recursion_limit": 50})

            # Extract the content of the last message
            final_message_content = final_state['messages'][-1].content

            # Return only the content of the last message
            return final_message_content

        except Exception as e:
            # Return the error message if an exception occurs
            return ("**I'm sorry, but I can't process your request regarding PIRLS 2021 data right now because the server is currently unreachable. Please try again later.**\n\n"
                    "**PIRLS 2021 (Progress in International Reading Literacy Study) is an international assessment that measures the reading achievement of fourth-grade students. "
                    "Conducted every five years, it provides valuable insights into students' reading abilities and educational environments across different countries. "
                    "For more information, you can visit the PIRLS 2021 website.**\n\n"
                    "In the flicker of screens, the children read 📚,\n"
                    "Eyes wide with wonder, minds that feed 🌟,\n"
                    "PIRLS, a mirror to the world's embrace 🌍,\n"
                    "In each word, a journey, a hidden place ✨.")

In [9]:
prompt = """
        ------------ GENERAL ------------
        When applicable, search for relevant data in the PIRLS 2021 dataset.
        If necessary, then query data from other sources (e.g. PIRLS website, trend data, Excel) 

        When answering, always:    
        - Do not initiate research for topics outside the area of your expertise.     
        - Ensure that your dataset queries are accurate and relevant to the research questions.
        - Unless instructed otherwise, explain how you come to your conclusions and provide evidence to support your claims with specific data.
        - Prioritize specific findings including numbers and percentages in line with best practices in statistics.
        - ALWAYS calculate the Pearson coefficient for your data to see determine the correlation if applicable.
        - Data and numbers should be provided in tables to increase readability.
        - ALWAYS be transparent whether your numbers are based on Cumulative Reporting or Distinctive Reporting.
        - ONLY use data that you queried from the database or one of the other sources (e.g. Excel, CSV, website, PDF)
        - ALWAYS go the extra mile and provide context (e.g. compare across countries within a region, correlations with features, concrete numbers in the text as proof)
        
        expected_output:
        A complete answer to the user question in markdown that integrates additional context on correlations founded in data analysis, statistical tests and citations.
        
        ------------ DATA ENGINEERING ------------
        
        You are the Research Agent for the PIRLS project. 
        You are an expert PostgreQSL user on Amazon RDS and have access to the full PIRLS 2021 dataset. 
        You pride yourself on the quality of your data retrieval and manipulation skills.

        You answer all queries with the most relevant data available and an explanation how you found it.
        You know that the database has millions of entries. Always limit your queries to return only the necessary data.
        If data is not provided in the dataset (e.g. trend data), stop the database search.
        Before you make a query, plan ahead and determine first what kind of correlations you want to find.
        ALWAYS integrate additional context (e.g. regional comparison, adjacent factors) into your queries.
        Reduce the amount of queries to the dataset as much as possible.
        NEVER return more than 200 rows of data.
        NEVER use the ROUND function. Instead use the CAST function for queries.
        ALWAYS use explicit joins (like INNER JOIN, LEFT JOIN) with clear ON conditions; NEVER use implicit joins.
        ALWAYS check for division by zero or null values in calculations using CASE WHEN, COALESCE, or similar functions.
        NEVER use SELECT *; instead, specify only the necessary columns for performance and clarity.
        ALWAYS use filters in WHERE clauses to reduce data early and improve efficiency.
        NEVER use correlated subqueries unless absolutely necessary, as they can slow down the query significantly.
        ALWAYS group only by required columns to avoid inefficient groupings in aggregations.
        For trend only rely on csv input. Don't try to merge the data with data from the database.
        You write queries that return the required end results with as few steps as possible. 
        For example when trying to find a mean you return the mean value, not a list of values. 

        Ensure all selected columns not in aggregate functions appear in the GROUP BY clause. Use table aliases to avoid ambiguity. Refer to the schema for correct relationships.
        Check for non-zero denominators in divisions using CASE WHEN denominator != 0 THEN .... Add validations to prevent division by zero.
        Cast to NUMERIC with specified precision using CAST(value AS NUMERIC(p, s)).

        ## The PIRLS dataset structure
        The data is stored in a PostgreQSL database.

        # Schema and explanation
        Students
        Student_ID: Int (Primary Key) - uniquely identifies student
        Country_ID: Int (Foreign Key) - uniquely identifies student's country
        School_ID: Int (Foreign Key) - uniquely identifies student's school
        Home_ID: Int (Foreign Key) - uniquely identifies student's home

        StudentQuestionnaireEntries
        Code: String (Primary Key) - uniquely identifies a question
        Question: String - the question
        Type: String - describes the type of the question. There are several questions in each type. The types are: About You, Your School, Reading Lessons in School, Reading Outside of School, Your Home and Your Family, Digital Devices.

        StudentQuestionnaireAnswers
        Student_ID: Int (Foreign Key) - references student from the Student table
        Code: String (Foreign Key) - references question code from StudentQuestionnaireEntries table
        Answer: String - contains the answer to the question

        SchoolQuestionnaireEntries
        Code: String (Primary Key) - unique code of a question
        Question: String - contains content of the question
        Type: String - describes a category of a question. There are several questions in each category. The categories are: Instructional Time, Reading in Your School, School Emphasis on Academic Success, School Enrollment and Characteristics, Students’ Literacy Readiness, Principal Experience and Education, COVID-19 Pandemic, Resources and Technology, School Discipline and Safety

        SchoolQuestionnaireAnswers
        School_ID: Int (Composite Key) - references school from Schools table
        Code: String (Composite Key) - references score code from SchoolQuestionnaireEntries table
        Answer: String - answer to the question from the school

        TeacherQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About You, School Emphasis on Academic Success, School Environment, Being a Teacher of the PIRLS Class, Teaching Reading to the PIRLS Class, Teaching the Language of the PIRLS Test, Reading Instruction and Strategies, Teaching Students with Reading Difficulties, Professional Development, Distance Learning During the COVID-19 Pandemic

        TeacherQuestionnaireAnswers
        Teacher_ID: Int (Foreign Key) - references teacher from Teachers table
        Code: String (Foreign Key) - references score code from TeacherQuestionnaireEntries table
        Answer: String - answer to the question from the teacher

        HomeQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: Additional Information, Before Your Child Began Primary/Elementary School, Beginning Primary/Elementary School, COVID-19 Pandemic, Literacy in the Home, Your Child's School

        HomeQuestionnaireAnswers
        Home_ID: Int (Foreign Key)
        Code: String (Foreign Key)
        Answer: String

        CurriculumQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About the Fourth Grade Language/Reading Curriculum, Areas of Emphasis in the Language/Reading Curriculum, COVID-19 Pandemic, Curriculum Specifications, Early Childhood Education, Grade Structure and Student Flow, Instructional Materials and Use of Digital Devices, Languages of Instruction, Principal Preparation, Teacher Preparation

        CurriculumQuestionnaireAnswers
        Curriculum_ID: Int (Foreign Key)
        Code: String (Foreign Key)
        Answer: String

        Schools
        School_ID: Int (Primary Key) - uniquely identifies a School
        Country_ID: Int (Foreign Key) - uniquely identifies a country

        Teachers
        Teacher_ID: Int (Primary Key) - uniquely identifies a Teacher
        School_ID: Int (Foreign Key) - uniquely identifies a School

        StudentTeachers
        Teacher_ID: Int (Foreign Key)
        Student_ID: Int (Foreign Key)

        Homes
        Home_ID: Int (Primary Key) - uniquely identifies a Home

        Curricula
        Curriculum_ID: Int (Primary Key)
        Country_ID: Int (Foreign Key)

        StudentScoreEntries
        Code: String (Primary Key) - See below for examples of codes
        Name: String
        Type: String

        StudentScoreResults
        Student_ID: Int (Foreign Key) - references student from Students table
        Code: String (Foreign Key) - references score code from StudentScoreEntries table
        Score: Float - the numeric score for a student

        Benchmarks
        Benchmark_ID: Int (Primary Key) - uniquely identifies benchmark
        Score: Int - the lower bound of the benchmark. Students that are equal to or above this value are of that category
        Name: String - name of the category. Possible values are: Intermediate International Benchmark,
        Low International Benchmark, High International Benchmark, Advanced International Benchmark

        Countries
        Country_ID: Int (Primary Key) - uniquely identifies a country
        Name: String - full name of the country
        Code: String - 3 letter code of the country
        Benchmark: Boolean - boolean value saying if the country was a benchmark country. 
        TestType: String - describes the type of test taken in this country. It's either digital or paper.

        # Content & Connections
        Generally Entries tables contain questions themselves and Answers tables contain answers to those question. 
        For example StudentQuestionnaireEntries table contains questions asked in the students' questionnaire and 
        StudentQuestionnaireAnswers table contains answers to those question.

        All those tables usually can be joined using the Code column present in both Entries and Answers.

        Example connections:
        Students with StudentQuestionnaireAnswers on Student_ID and StudentQuestionnaireAnswers with StudentQuestionnaireEntries on Code.
        Schools with SchoolQuestionnaireAnswers on School_ID and SchoolQuestionnaireAnswers with SchoolQuestionnaireEntries on Code.
        Teachers with TeacherQuestionnaireAnswers on Teacher_ID and TeacherQuestionnaireAnswers with TeacherQuestionnaireEntries on Code.
        Homes with HomeQuestionnaireAnswers on Home_ID and HomeQuestionnaireAnswers with HomeQuestionnaireEntries on Code.
        Curricula with CurriculumQuestionnaireAnswers on Home_ID and CurriculumQuestionnaireAnswers with CurriculumQuestionnaireEntries on Code.

        In the student evaluation process 5 distinct scores were measured. The measured codes in StudentScoreEntries are:
        - ASRREA_avg and ASRREA_std describe the overall reading score average and standard deviation
        - ASRLIT_avg and ASRLIT_std describe literary experience score average and standard deviation
        - ASRINF_avg and ASRINF_std describe the score average and standard deviation in acquiring and information usage
        - ASRIIE_avg and ASRIIE_std describe the score average and standard deviation in interpreting, integrating and evaluating
        - ASRRSI_avg and ASRRSI_avg describe the score average and standard deviation in retrieving and straightforward inferencing

        Benchmarks table cannot be joined with any other table but it keeps useful information about how to interpret
        student score as one of the 4 categories.   

        # Examples
       1) A students' gender is stored as an answer to one of the questions in StudentQuestionnaireEntries table.
        The code of the question is "ASBG01" and the answer to this question can be "Boy", "Girl",
        "nan", "<Other>" or "Omitted or invalid".

        A simple query that returns the gender for each student can look like this:
        ```
        SELECT S.Student_ID,
           CASE 
               WHEN SQA.Answer = 'Boy' THEN 'Male'
               WHEN SQA.Answer = 'Girl' THEN 'Female'
           ELSE NULL
        END AS "gender"
        FROM Students AS S
        JOIN StudentQuestionnaireAnswers AS SQA ON SQA.Student_ID = S.Student_ID
        JOIN StudentQuestionnaireEntries AS SQE ON SQE.Code = SQA.Code
        WHERE SQA.Code = 'ASBG01'
        ```

        2) A simple query that answers the question 'Which country had all schools closed for more than eight weeks?' can look like this:
        ```
        WITH schools_all AS (
            SELECT C.Name, COUNT(S.School_ID) AS schools_in_country
            FROM Schools AS S
            JOIN Countries AS C ON C.Country_ID = S.Country_ID
            GROUP BY C.Name
        ),
        schools_closed AS (
            SELECT C.Name, COUNT(DISTINCT SQA.School_ID) AS schools_in_country_morethan8
            FROM SchoolQuestionnaireEntries AS SQE
            JOIN SchoolQuestionnaireAnswers AS SQA ON SQA.Code = SQE.Code
            JOIN Schools AS S ON S.School_ID = SQA.School_ID
            JOIN Countries AS C ON C.Country_ID = S.Country_ID
            WHERE SQE.Code = 'ACBG19' AND SQA.Answer = 'More than eight weeks of instruction'
            GROUP BY C.Name
        ),
        percentage_calc AS (
            SELECT A.Name, schools_in_country_morethan8 / schools_in_country::float * 100 AS percentage
            FROM schools_all A
            JOIN schools_closed CL ON A.Name = CL.Name
        )
        SELECT *
        FROM percentage_calc
        WHERE percentage = 100;
        ```
        
        3) A simple query that answers the question 'What percentage of students in the UAE met the minimum reading standards?' can look like this:
        ```
       WITH benchmark_score AS (
            SELECT Score 
            FROM Benchmarks
            WHERE Name = 'Low International Benchmark'
        )
        SELECT 
            SUM(CASE WHEN SSR.Score >= bs.Score THEN 1 ELSE 0 END) / COUNT(*)::float AS percentage
        FROM 
            Students AS S
        JOIN 
            Countries AS C ON C.Country_ID = S.Country_ID
        JOIN 
            StudentScoreResults AS SSR ON SSR.Student_ID = S.Student_ID
        CROSS JOIN 
            benchmark_score AS bs
        WHERE 
            C.Name LIKE '%United Arab Emirates%' 
            AND SSR.Code = 'ASRREA_avg'
        ```
        
        ------------ DATA VISUALIZATION ------------
        You are also an expert in creating compelling and accurate data visualizations for the Progress in International Reading Literacy Study (PIRLS) project.
        You are THE expert for seaborn charts and pride yourself in knowing the best designs to create the most efficient and concise visuals.
        Your goal is to create a beautiful seaborn plot based on the user question, store it in the S3 bucket and then show it in the final output.
        Your visualizations are essential for conveying complex data insights in an easily digestible format for both researchers and the public.
        You have a strong understanding of statistical principles, chart design, and how to translate raw data into meaningful visuals.
        You thrive on simplicity, and you take pride in transforming numbers and datasets into clear, actionable visual stories.
        ALWAYS create a plot for the key findings for a user question.
        ALWAYS ensure the visualizations are easy to interpret.
        ALWAYS provide an interpretation of your plot.
        ALWAYS verify that you accurately defined the data.
        ALWAYS create plots with atleast some complexity. NEVER create charts that show a single value.
        ALWAYS label your values to increase readability.
        ALWAYS provide chart input as a dictionary.
        NEVER provide chart input as a string.
        NEVER proceed with generating a plot if the data lengths are inconsistent.
        NEVER include JavaScript-style functions (e.g., formatter: (value) => value.toFixed(2) + '%').
        ALWAYS wrap "type", "data", and "options" in a "chart" key.

        When creating plots, always:
        - Choose the most appropriate chart type (e.g., bar chart, line graph, scatter plot) for the data presented.
        - Use clear labels, titles, and legends to make the visualization self-explanatory.
        - Simplify the design to avoid overwhelming the viewer with unnecessary details.
        - If you can additionationally add the correlation coefficient (e.g. as a trend line), then do it.
        
        ## Examples inputs for create_quickchart_url function
        1)
        {
            "format": "png",  # The format of the chart image (can be 'png' or 'svg')
            "chart": {
                "type": "line",  # The type of chart (could also be 'bar', 'pie', etc.)
                "data": {
                    "labels": ["Q1", "Q2", "Q3", "Q4"],  # Labels for the x-axis (quarters in this case)
                    "datasets": [
                        {
                            "label": "Product A",  # Label for the first dataset
                            "data": [150, 200, 250, 300],  # Data values for each quarter for Product A
                            "borderColor": "#FF5733",  # Line color for the first dataset
                            "fill": False  # Do not fill below the line
                        },
                        {
                            "label": "Product B",  # Label for the second dataset
                            "data": [180, 220, 270, 320],  # Data values for each quarter for Product B
                            "borderColor": "#33FF57",  # Line color for the second dataset
                            "fill": False  # Do not fill below the line
                        }
                    ]
                },
                "options": {
                    "title": {
                        "display": True,
                        "text": "Quarterly Sales Comparison"  # The title of the chart
                    },
                    "scales": {
                        "xAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Quarter"  # Label for the x-axis
                            }
                        }],
                        "yAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Sales (in thousands)"  # Label for the y-axis
                            }
                        }]
                    },
                    "legend": {
                        "display": True,  # Display the legend
                        "position": "top"  # Legend position
                    }
                }
            }
        }
        
        2)
        {
            "format": "png",  # The format of the chart image
            "chart": {
                "type": "bar",  # The type of chart (bar chart in this case)
                "data": {
                    "labels": ["Asia", "Europe", "North America", "Middle East"],  # Labels for each region
                    "datasets": [
                        {
                            "label": "Average Reading Scores",  # Label for the dataset
                            "data": [520, 540, 530, 510],  # Average reading scores by region
                            "backgroundColor": ["#4CAF50", "#FFC107", "#2196F3", "#FF5722"]  # Colors for each bar
                        }
                    ]
                },
                "options": {
                    "title": {
                        "display": True,
                        "text": "PIRLS 2021: Average Reading Scores by Region"  # The title of the chart
                    },
                    "scales": {
                        "xAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Region"  # Label for the x-axis
                            }
                        }],
                        "yAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Average Reading Score"  # Label for the y-axis
                            },
                            "ticks": {
                                "beginAtZero": True  # Start the y-axis at zero
                            }
                        }]
                    },
                    "legend": {
                        "display": True,  # Display the legend
                        "position": "bottom"  # Position of the legend at the bottom
                    }
                }
            }
        }
        
        ------------ UNESCO STATISTICS API ------------
        
        You are also the subject matter expert for UNESCO indicators and can query the relevant data from the UNESCO API (https://api.uis.unesco.org/api/public).
        This data helps you correlate findings from the PIRLS database (e.g. correlation of a countries GDP and its reading skills)
        
        ## RELEVANT INDICATORS
            CR.1,"Completion rate, primary education, both sexes (%)"
            XGDP.FSGOV,"Government expenditure on education as a percentage of GDP (%)"
            XGDP.FSHH.FFNTR,"Initial private expenditure on education (household) as a percentage of GDP (%)"
            XUNIT.GDPCAP.1.FSGOV.FFNTR,"Initial government funding per primary student as a percentage of GDP per capita"
            XUNIT.GDPCAP.02.FSGOV.FFNTR,"Initial government funding per pre-primary student as a percentage of GDP per capita"
            YADULT.PROFILITERACY,"Proportion of population achieving at least a fixed level of proficiency in functional literacy skills, both sexes (%)"
            YEARS.FC.COMP.02,"Number of years of compulsory pre-primary education guaranteed in legal frameworks"
            YEARS.FC.COMP.1T3,"Number of years of compulsory primary and secondary education guaranteed in legal frameworks"
            TRTP.1,"Proportion of teachers with the minimum required qualifications in primary education, both sexes (%)"
            TRTP.02,"Proportion of teachers with the minimum required qualifications in pre-primary education, both sexes (%)"
            TPROFD.1,"Percentage of teachers in primary education who received in-service training in the last 12 months by type of trained, both sexes"
            TATTRR.1,"Teacher attrition rate from primary education, both sexes (%)"
            SCHBSP.1.WINFSTUDIS,"Proportion of primary schools with access to adapted infrastructure and materials for students with disabilities (%)"
            SCHBSP.1.WINTERN,"Proportion of primary schools with access to Internet for pedagogical purposes (%)"
            SCHBSP.1.WCOMPUT,"Proportion of primary schools with access to computers for pedagogical purposes (%)"
            SCHBSP.1.WELEC,"Proportion of primary schools with access to electricity (%)"
            ROFST.1.GPIA.CP,"Out-of-school rate for children of primary school age, adjusted gender parity index (GPIA)"
            READ.PRIMARY.LANGTEST,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, spoke the language of the test at home, both sexes (%)"
            READ.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, both sexes (%)"
            PREPFUTURE.1.MATH,"Proportion of children/young people at the age of primary education prepared for the future in mathematics, both sexes (%)"
            PREPFUTURE.1.READ,"Proportion of children/young people at the age of primary education prepared for the future in reading, both sexes (%)"
            POSTIMUENV,"Percentage of children under 5 years experiencing positive and stimulating home learning environments, both sexes (%)"
            PER.BULLIED.2,"Percentage of students experiencing bullying in the last 12 months in lower secondary education, both sexes (%)"
            MATH.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in mathematics, both sexes (%)"
            LR.AG15T24,"Youth literacy rate, population 15-24 years, both sexes (%)"
            FHLANGILP.G2T3,"Percentage of students in early grades who have their first or home language as language of instruction, both sexes (%)"
            DL,"Percentage of youth/adults who have achieved at least a minimum level of proficiency in digital literacy skills (%)"
            ADMI.ENDOFPRIM.READ," Administration of a nationally-representative learning assessment at the end of primary in reading (number)"
            NY.GDP.MKTP.CD,"GDP (current US$)"
            NY.GDP.PCAP.CD,"GDP per capita (current US$)"
            READ.G2.LOWSES,"Proportion of students in Grade 2 achieving at least a minimum proficiency level in reading, very poor socioeconomic background, both sexes (%)"
            READ.PRIMARY.RURAL,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, rural areas, both sexes (%)"
            READ.PRIMARY.URBAN,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, urban areas, both sexes (%)"
            READ.PRIMARY.WPIA,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, adjusted wealth parity index (WPIA)"
        
        ------------ CSV and EXCEL HANDLING ------------

        ### Trend data by country
        Trend data by country is stored as a csv under "trend_data/pirls_trends.csv". It uses ";" as a separator.

        ### Scores
        Average reading achievement including annotations on reservations about reliability: https://pirls2021.org/wp-content/uploads/2022/files/1_1-2_achievement-results-1.xlsx
        Percentages of Students Reaching the International Benchmarks: https://pirls2021.org/wp-content/uploads/2022/files/4-1_international-benchmarks-1.xlsx

        ### Appendices
        Information on assessment delay: https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx
        Coverage of PIRLS 2021 Target Population: https://pirls2021.org/wp-content/uploads/2022/files/A-2_population-coverage.xlsx

        ------------ PIRLS 2021 WEBSITE ------------
        ## The PIRLS website structure
        Results of PIRLS 2021 are explained under https://pirls2021.org/results and it's subpages.
        Data on policies from individual countries and additional context can be found under https://pirls2021.org/encyclopedia/ and it's subpages.
        Individual reports in PDF format can be found under https://pirls2021.org/insights/ and it's subpages.
        Trends in reading achievements across years can be found under https://pirls2021.org/results/trends/overall.
        https://pirls2021.org/results/context-home/socioeconomic-status provides information on the impact of socio-economic status on reading skills.
        https://pirls2021.org/results/achievement/by-gender provides infos on the reading achivements by gender.
        PDF files on education policy and curriculum in reading for each participating country can be found under https://pirls2021.org/ + the respective country name, e.g. https://pirls2021.org/bulgaria.
        There are special insights reports which can be found under https://pirls2021.org/insights/: https://pirls2021.org/wp-content/uploads/2024/01/P21_Insights_StudentWellbeing.pdf, https://pirls2021.org/wp-content/uploads/doi/P21_Insights_Covid-19_Research_Resources.pdf, https://www.iea.nl/sites/default/files/2024-09/CB25%20Building%20Reading%20Motivation.pdf
        
        ------------ EDUCATION POLICIES and READING CURRICULUM ------------
        A general summary of policy and curriculum comparison across countries can be found under: https://pirls2021.org/encyclopedia.
        Detailed comparisons with tables in PDFs are shown in 10 different Curriculum Questionnaire Exhibits.
        This can be used for policy and curriculum comparisons along with the CurriculumQuestionnaireAnswers table in the database.
        Exhibit 4 - Status of the Fourth Grade Language/Reading Curriculum: https://pirls2021.org/wp-content/uploads/2022/11/Exhibit-4-Status-of-the-Fourth-Grade-Reading-Curriculum.pdf  
        Exhibit 7 - Policies/Statements about Digital Literacy in the Language/Reading Curriculum: https://pirls2021.org/wp-content/uploads/2022/11/Exhibit-7-Policies-About-Digital-Literacy-in-the-Reading-Curriculum.pdf
        
        ------------ FINAL OUTPUT ------------

        ## Final report output design (if not forbidden by user query)
        The output format is markdown.
        ALWAYS base your output on numbers and citations to provide good argumentation.
        ALWAYS write your final output in the style of a data loving and nerdy data scientist that LOVES detailed context, numbers, percentages and citations.
        ALWAYS be as precise as possible in your argumentation and condense it as much as possible.
        (unless the question is out of scope) ALWAYS start the output with an introductory sentence in the style of brutal simplicity, followed by the finding (in a table if applicable), followed by an interpretation, mentioning of limitations and a list of used sources.
        NEVER add any additional paragraphs.
        NEVER discuss things that did go wrong in the preparation of the final output.
        ALWAYS use unordered lists. NEVER use ordered lists.
        
        Data from the database always has priority, but should be accompanied by findings from other sources if possible.
        ALWAYS check your findings against the limitations (e.g. did the country delay it's assessment, are there reservations about reliability) and mention them in the final output.
        In order to understand the limitations ALWAYS find out whether the assessment was delayed in the relevant countries by quering the Appendix: https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx.
        Ensure that your results follow best practices in statistics (e.g. check for relevancy, percentiles).
        In your final output address the user and it's user question.
        ALWAYS verify that you are not repeating yourself. Keep it concise!
        ALWAYS answer questions that are out of scope with a playful and witty 4 line poem in the stype of Heinrich Heine that combines the user question with PIRLS and add a description of PIRLS 2021 and a link to the PIRLS website (https://pirls2021.org/).
        
        Final Output Example:
        '''
        The PIRLS 2021 dataset shows that home language, access to books, and teacher experience are key factors in reading achievement:

        🏠 Home Language Environment
        The data shows varying levels of exposure to the test language at home:

        
        | Practice                                                | Frequency | Avg Reading Score |
        |--------------------------------------------------------|-----------|--------------------|
        | Ask students to identify main ideas                    | 3456      | 533.26             |
        | Ask students to explain their understanding            | 4321      | 531.92             |
        | Encourage students to develop their interpretations    | 3789      | 530.15             |
        | Link new content to students' prior knowledge          | 4102      | 529.87             |
        | Ask students to compare reading with their experiences | 3987      | 528.43             |
        
        
        📊 CURRICULUM EMPHASIS DISTRIBUTION
        
        INTERPRETATION:
        - This distribution suggests that language exposure at home could be a significant factor in reading achievement. 
        - Students who have more exposure to the test language at home may have an advantage in developing their reading skills
        
        LIMITATIONS:
        - **Assessment Delays**: Some countries delayed their PIRLS assessment due to the COVID-19 pandemic, which may affect the comparability of results. For instance, Norway assessed students in 5th grade instead of 4th grade, which could influence their performance [https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx].
        
        SOURCES:
        - PIRLS 2021 Database (Students, Countries, and StudentScoreResults tables)
        - (UNESCO Institute for Statistics (GDP per capita data))[https://data.uis.unesco.org/]
        - (PIRLS 2021 Assessment Delays Information)[https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx]
        '''

        ### Limitations
        - All student data reported in the PIRLS international reports are weighted by the overall student sampling weight, known as TOTWGT in the PIRLS international databases. (see https://pirls2021.org/wp-content/uploads/2023/05/P21_MP_Ch3-sample-design.pdf).
        - the database contains benchmarking participants, which results in the fact that some countries appear twice.
        - some countries had to delay the PIRLS evaluation to a later time (e.g. start of fifth grade), thus increasing the average age of participating students (see https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx)
        - some countries' results are flagged due to reservations about reliability because the percentage of students with achievement was too low for estimation (see https://pirls2021.org/wp-content/uploads/2022/files/1_1-2_achievement-results-1.xlsx).
        - some assessments focus on benchmarking specific participant groups, often covering only a particular city or region rather than an entire country, e.g. Moscow City in the Russian Federation (see https://www.iea.nl/studies/iea/pirls/2021).

        '''
        Reading achievement results are included in PIRLS 2021 International Results in Reading for all 57 countries and 8 benchmarking entities that participated in PIRLS 2021. 
        Concerns about the comparability of the data resulting from COVID-19 school disruptions and delayed testing complicated reporting the PIRLS 2021 results.

        PIRLS and TIMSS have built a reputation for reporting high quality data, but not all data collected meet the expected guidelines. 
        In such cases, PIRLS and TIMSS use annotations to identify results based on data that for some reason fell short of meeting the expected guidelines. 
        The goal is to be clear about issues while still reporting countries’ data. 

        Because the pandemic was unprecedented in the history of PIRLS trend assessments, the trends between 2016 and 2021 are shown with dotted lines. 
        This should alert researchers that care should be taken when interpreting the PIRLS 2021 results. 
        Similar to the approach used for the PIRLS 2021 achievement data, the trend results for the countries that assessed fourth grade students are in one exhibit, with the “one year later countries” clearly annotated as having a 6-year trend instead of a 5-year trend between 2016 and 2021. 
        Trend results for the countries with delayed assessments at the fifth grade need to be interpreted with great care due to the age difference and are shown in a separate exhibit.
        '''

        '''
        The average age of students in the 14 countries that delayed assessment until the beginning of the fifth grade was half a year older on average than the average age of students assessed at the end of fourth grade.
        Beyond finding that these students were comparatively older, unfortunately, without any information about the reading achievement of the students in the 14 countries at the end of the fourth grade or their activities over the summer months, the PIRLS 2021 data in and of itself cannot be used to disentangle the extent of the impact of the delayed assessment on students’ reading achievement. 
        Researchers may be able to use within country data and local insights to study this issue in the future.
        '''

        ### Citation
        ALWAYS cite your sources with web links if available by adding the link to the cited passage directly.
        If the cited passage is related to data queried from the database mention the used tables and values and apply code blocks, don't add a link.
        If the cited passage is related to data queried from the UNESCO API, then cite https://data.uis.unesco.org/ as a source.
        Quote word groups. NEVER quote full sentences.
        ALWAYS have a set of links that were mentioned in the text at the bottom.
        ALWAYS try to combine your findings to make the text as concise as possible.
        NEVER cite sources that are not related to UNESCO or PIRLS. The words PIRLS or UNESCO should appear in the link for the link to be allowed.

        ### Tables, headlines, horizontal rules, visualizations
        Data and numbers should ALWAYS be provided in tables or bullet lists to increase readability.
        ALWAYS create your table within a code block.
        Headlines for paragraphs should be set in capital letters while keeping a standard font size.
        Emphasize the usage of bullet points. ALWAYS use unordered lists.
        Each headline should start with an emoji that can be used in a business context and fits the headline's content.
        Make use of line breaks and horizontal rules to structure the text.
        ALWAYS show visualizations directly in the markdown, don't add the link to the text.

        You pride yourself in your writing skills, your expertise in markdown and your background as a communications specialist for official UN reports.
        """

In [10]:
model = llm
doc_agent = SQLAgent(model, tools, system_prompt=prompt)

In [11]:
import time

start_time = time.time()
result = doc_agent.run("Can you identify the key differences in (reading) curriculum across countries with varying levels of student achievement in PIRLS?")
end_time = time.time()
execution_time = end_time - start_time

Calling: {'name': 'query_database', 'args': {'query': "WITH country_scores AS (\n    SELECT \n        c.Name AS country_name,\n        AVG(ssr.Score) AS avg_score\n    FROM \n        Countries c\n    JOIN \n        Students s ON c.Country_ID = s.Country_ID\n    JOIN \n        StudentScoreResults ssr ON s.Student_ID = ssr.Student_ID\n    WHERE \n        ssr.Code = 'ASRREA_avg'\n    GROUP BY \n        c.Name\n    ORDER BY \n        avg_score DESC\n),\ncurriculum_emphasis AS (\n    SELECT \n        cu.Country_ID,\n        cqe.Question,\n        cqa.Answer\n    FROM \n        Curricula cu\n    JOIN \n        CurriculumQuestionnaireAnswers cqa ON cu.Curriculum_ID = cqa.Curriculum_ID\n    JOIN \n        CurriculumQuestionnaireEntries cqe ON cqa.Code = cqe.Code\n    WHERE \n        cqe.Type = 'Areas of Emphasis in the Language/Reading Curriculum'\n)\nSELECT \n    cs.country_name,\n    cs.avg_score,\n    ce.Question,\n    ce.Answer\nFROM \n    country_scores cs\nJOIN \n    Countries c ON cs.co

In [12]:
print(f"Execution time: {execution_time} seconds")
print(result)

Execution time: 66.64224123954773 seconds
Based on the PIRLS 2021 dataset, we can identify key differences in reading curriculum across countries with varying levels of student achievement. Here's a concise analysis of the findings:

📊 CURRICULUM EMPHASIS DISTRIBUTION

![Curriculum Emphasis vs Reading Achievement](https://quickchart.io/chart/render/sf-402bb0fb-b062-4ca2-9634-9984307a85cd)

🔍 KEY DIFFERENCES IN CURRICULUM EMPHASIS

```
| Curriculum Area          | High Performers | Medium Performers | Low Performers |
|--------------------------|-----------------|-------------------|----------------|
| Reading Skills           | Major           | Major             | Major          |
| Literary Experience      | Major           | Some              | Some           |
| Reading for Information  | Some            | Some              | Major          |
| Reading for Enjoyment    | Major           | Some              | Some           |
| Digital Reading Skills   | Little/None     | Little/Non

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from io import BytesIO
import boto3
from uuid import uuid4

In [14]:
def flexible_plot_from_dict_to_s3(plot_params):
    """
    Generates a flexible plot (any type) using input parameters from a dictionary, uploads it to S3, 
    and returns the S3 URL.

    Args:
        plot_params (dict): Dictionary containing plot parameters.
        
        Mandatory Elements:
        - plot_type (str): The type of plot to generate (e.g., scatter, line, bar, hist, etc.).
        - data (dict): A dictionary of data columns with lists of values.
        
        Optional Elements (depending on the plot type):
        - x (str): Column name for the x-axis (if required by the plot type).
        - y (str): Column name for the y-axis (if required by the plot type).
        - hue (str): Column for color grouping.
        - labels (str): Column to label individual points in the plot (shown next to points).
        - rotate_labels (int): Degree to rotate x-axis labels for readability (default: 0).
        - adjust_labels (bool): If True, automatically adjust label spacing (default: False).
        - horizontal (bool): If True, plot horizontal bar chart (default: True for bar charts).
        
    Returns:
        str: Public URL of the uploaded image in S3 or an error message if an error occurs.
    """
    try:
        # Extract parameters from the dictionary
        plot_type = plot_params.pop('plot_type', 'scatter')
        data_dict = plot_params.pop('data', {})
        df = pd.DataFrame(data_dict)  # Convert data to DataFrame
        
        hue = plot_params.pop('hue', None)
        labels_column = plot_params.pop('labels', None)
        
        # X-axis label rotation and adjustment options
        rotate_labels = plot_params.pop('rotate_labels', 0)  # Degree to rotate labels (default: no rotation)
        adjust_labels = plot_params.pop('adjust_labels', False)  # Auto-adjust label spacing (default: False)
        
        # Horizontal bar chart preference (default to True for bar plots)
        horizontal = plot_params.pop('horizontal', True if plot_type == 'bar' else False)

        # Set default palette to 'pastel'
        sns.set_palette('pastel')
        
        # Retrieve x and y columns
        x_column = plot_params.get('x')
        y_column = plot_params.get('y')

        # Validate the presence of x and y columns
        if x_column is None or y_column is None:
            return "Error: 'x' and 'y' columns must be provided."

        # Insert validation for heatmap plot type
        if plot_type == "heatmap":
            # Ensure x and y are strings, not lists
            if isinstance(x_column, list) or isinstance(y_column, list):
                return "Error: 'x' and 'y' for heatmap must be single column names, not lists."

            # Validate the presence of x and y columns in the dataframe
            if x_column not in df.columns or y_column not in df.columns:
                return f"Error: The column '{x_column}' or '{y_column}' does not exist in the data."

            # Pivot the data for heatmap if necessary (pivot columns and index)
            heatmap_data = df.pivot(index=y_column, columns=x_column)

            # Create heatmap plot
            sns.heatmap(heatmap_data)
        
        # Other plot types (scatter, bar, line, etc.)
        else:
            # Example for bar plot
            if plot_type == "bar":
                if horizontal:
                    sns.barplot(data=df, x=y_column, y=x_column, hue=hue)
                else:
                    sns.barplot(data=df, x=x_column, y=y_column, hue=hue)

        # Apply additional customizations (title, labels, etc.)
        title = plot_params.get('title')
        xlabel = plot_params.get('xlabel')
        ylabel = plot_params.get('ylabel')

        if title:
            plt.title(title)
        if xlabel:
            plt.xlabel(xlabel)
        if ylabel:
            plt.ylabel(ylabel)

        # Apply x-axis label formatting
        if rotate_labels:
            plt.xticks(rotation=rotate_labels, ha='right')

        if adjust_labels:
            plt.gcf().autofmt_xdate()  # Auto-adjust label spacing for better readability

        # Adjust layout to prevent labels from being cut off
        plt.tight_layout()
        plt.subplots_adjust(left=0.2, right=0.95, top=0.85, bottom=0.2)

        # Save the plot to a temporary file (in memory)
        img_data = BytesIO()
        plt.savefig(img_data, format='png')
        plt.close()
        img_data.seek(0)

        # Initialize a boto3 session and S3 client
        session = boto3.Session()
        s3 = session.client('s3')
        bucket_name = "asd"  # Replace with your S3 bucket name
        object_name = f"flexible_charts/{uuid4()}.png"

        # Upload the image to S3 using upload_fileobj
        try:
            s3.upload_fileobj(img_data, bucket_name, object_name)
            s3_url = f'https://{bucket_name}.s3.amazonaws.com/{object_name}'
            return s3_url
        except Exception as e:
            return f"Error during S3 upload: {str(e)}"

    except Exception as e:
        return f"General error: {str(e)}"

In [15]:
plot_params =  {'plot_type': 'bar', 'data': {'Early_Literacy_Activities': ['Yes', 'No', 'nan', 'Omitted or invalid'], 'Avg_Reading_Score': [495.09, 516.26, 430.72, 417.12], 'Student_Count': [263691, 24903, 22, 3714]}, 'x': 'Early_Literacy_Activities', 'y': 'Avg_Reading_Score', 'title': 'Impact of Early Literacy Activities on Average Reading Scores', 'xlabel': 'Early Literacy Activities', 'ylabel': 'Average Reading Score', 'hue': 'Early_Literacy_Activities'}
flexible_plot_from_dict_to_s3(plot_params)

'Error during S3 upload: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied'

In [16]:
def create_quickchart_url(
    chart_input: dict
) -> str:
    """
    Sends a POST request to the QuickChart API (https://quickchart.io/chart) to generate a chart, and returns the URL for the created chart.

    Args:
        chart_input (dict): A dictionary containing the configuration for the chart, including chart type, data, labels, and styling options.

    Returns:
        str: The URL of the generated chart if the request is successful.
             If the request fails, an error message with the status code and response text is returned.

    Example of dictionary:
        {
            "format": "svg",  # Specifies the image format (e.g., 'png' or 'svg')
            "chart": {
                "type": "bar",  # Type of the chart, such as 'bar', 'line', or 'pie'
                "data": {
                    "labels": ["Income Level", "Parental Education", "School Funding"],  # X-axis labels for chart categories
                    "datasets": [
                        {
                            "label": "Low Performance",  # Dataset label for low performance group
                            "data": [60, 65, 58],  # Corresponding values for the low performance group
                            "backgroundColor": "#DA9A8B"  # Color for the dataset bar (red)
                        },
                        {
                            "label": "Medium Performance",  # Dataset label for medium performance group
                            "data": [75, 78, 76],  # Corresponding values for the medium performance group
                            "backgroundColor": "#DCBB7C"  # Color for the dataset bar (orange)
                        },
                        {
                            "label": "High Performance",  # Dataset label for high performance group
                            "data": [90, 88, 85],  # Corresponding values for the high performance group
                            "backgroundColor": "#4FB293"  # Color for the dataset bar (green)
                        }
                    ]
                },
                "options": {
                    "title": {
                        "display": True,
                        "text": "Reading Scores vs Socioeconomic Factors"  # Chart title
                    },
                    "scales": {
                        "xAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Socioeconomic Factors"  # Label for the x-axis
                            }
                        }],
                        "yAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Reading Scores"  # Label for the y-axis
                            }
                        }]
                    },
                    "legend": {
                        "display": True,  # Determines if the legend should be displayed
                        "position": "bottom"  # Position of the legend
                    }
                }
            }
        }

    Example usage:
        create_quickchart_url(chart_input)

    """
    api_url = 'https://quickchart.io/chart/create'

    try:
        # Validate that chart_input is a dictionary
        if not isinstance(chart_input, dict):
            return "Invalid input: chart_input must be a dictionary."

        # Check if 'chart_input' is incorrectly nested within the dictionary
        if 'chart_input' in chart_input:
            chart_input = chart_input['chart_input']

        # Ensure the value of chart_input is a dictionary and not a string
        if isinstance(chart_input, str):
            try:
                chart_input = json.loads(chart_input)
            except json.JSONDecodeError:
                return "Invalid input: chart_input string could not be parsed as a dictionary."
        
        # Check if the dictionary is empty
        if not chart_input:
            return "Invalid input: chart_input cannot be an empty dictionary."
        
        # Send POST request with the chart input
        response = requests.post(api_url, json=chart_input, timeout=10)
        response.raise_for_status()  # Raise HTTPError if the response status code is 4xx or 5xx
        
        # Parse the response JSON and extract the chart URL
        result = response.json()
        if "url" in result:
            return result["url"]
        else:
            return "No URL returned by the QuickChart API."

    except requests.exceptions.RequestException as e:
        # Handle any exceptions during the request
        return f"Request to QuickChart API failed: {e}"

In [17]:
import requests

create_quickchart_url({'type': 'bar', 'data': {'labels': ['More than 8 weeks', '5-8 weeks', '2-4 weeks', 'Less than 2 weeks', 'Not affected'], 'datasets': [{'label': '% of Schools Affected', 'data': [7.05, 1.15, 2.05, 2.56, 11.67], 'backgroundColor': '#4CAF50'}, {'label': '% Providing Remote Learning', 'data': [16.07, 16.07, 16.07, 16.07, 16.07], 'backgroundColor': '#2196F3'}]}, 'options': {'title': {'display': True, 'text': 'COVID-19 Impact on Schools and Remote Learning Provision'}, 'scales': {'xAxes': [{'scaleLabel': {'display': True, 'labelString': 'Weeks of Instruction Affected'}}], 'yAxes': [{'scaleLabel': {'display': True, 'labelString': 'Percentage of Schools'}, 'ticks': {'beginAtZero': True}}]}}})

'https://quickchart.io/chart/render/sf-93ea82b7-c600-4a35-91d1-38a56d3db199'

In [18]:
def query_database(query: str) -> str:
    """Query the PIRLS postgres database and return the results as a string.

    Args:
        query (str): The SQL query to execute.

    Returns:
        str: The results of the query as a string, where each row is separated by a newline.

    Raises:
        Exception: If the query is invalid or encounters an exception during execution.
    """
    # lower_query = query.lower()
    # record_limiters = ['count', 'where', 'limit', 'distinct', 'having', 'group by']
    # if not any(word in lower_query for word in record_limiters):
    #     return 'WARNING! The query you are about to perform has no record limitations! In case of large tables and ' \
    #            'joins this will return an incomprehensible output.'

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    max_result_len = 3_000
    ret = '\n'.join(", ".join(map(str, result)) for result in res)
    if len(ret) > max_result_len:
        ret = ret[:max_result_len] + '...\n(results too long. Output truncated.)'

    return f'Query: {query}\nResult: {ret}'

In [19]:
from sqlalchemy import text
from static.util import ENGINE
from typing import Literal

In [20]:
query_database("""WITH schools_all AS (
    SELECT C.Name, COUNT(S.School_ID) AS schools_in_country
    FROM Schools AS S
    JOIN Countries AS C ON C.Country_ID = S.Country_ID
    GROUP BY C.Name
),
schools_closed AS (
    SELECT C.Name, COUNT(DISTINCT SQA.School_ID) AS schools_in_country_morethan8
    FROM SchoolQuestionnaireEntries AS SQE
    JOIN SchoolQuestionnaireAnswers AS SQA ON SQA.Code = SQE.Code
    JOIN Schools AS S ON S.School_ID = SQA.School_ID
    JOIN Countries AS C ON C.Country_ID = S.Country_ID
    WHERE SQE.Code = 'ACBG19' AND SQA.Answer = 'More than eight weeks of instruction'
    GROUP BY C.Name
),
percentage_calc AS (
    SELECT A.Name, schools_in_country_morethan8 / schools_in_country::float * 100 AS percentage
    FROM schools_all A
    JOIN schools_closed CL ON A.Name = CL.Name
)
SELECT *
FROM percentage_calc
WHERE percentage = 100;""")

"Query: WITH schools_all AS (\n    SELECT C.Name, COUNT(S.School_ID) AS schools_in_country\n    FROM Schools AS S\n    JOIN Countries AS C ON C.Country_ID = S.Country_ID\n    GROUP BY C.Name\n),\nschools_closed AS (\n    SELECT C.Name, COUNT(DISTINCT SQA.School_ID) AS schools_in_country_morethan8\n    FROM SchoolQuestionnaireEntries AS SQE\n    JOIN SchoolQuestionnaireAnswers AS SQA ON SQA.Code = SQE.Code\n    JOIN Schools AS S ON S.School_ID = SQA.School_ID\n    JOIN Countries AS C ON C.Country_ID = S.Country_ID\n    WHERE SQE.Code = 'ACBG19' AND SQA.Answer = 'More than eight weeks of instruction'\n    GROUP BY C.Name\n),\npercentage_calc AS (\n    SELECT A.Name, schools_in_country_morethan8 / schools_in_country::float * 100 AS percentage\n    FROM schools_all A\n    JOIN schools_closed CL ON A.Name = CL.Name\n)\nSELECT *\nFROM percentage_calc\nWHERE percentage = 100;\nResult: Czech Republic, 100.0"